In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# plotly display option
import plotly.io as pio
pd.options.plotting.backend = 'plotly'
pio.renderers.default = "notebook_connected"
pd.options.display.max_rows = 100

# 수입자동차 Model별 데이터

### Data preprocessing
- total row 삭제
- object -> int 변경
- Rand Rover 브랜드명 통일

In [1]:
model_df = pd.read_csv("./datas/model.csv")
model_df = model_df.drop(model_df[model_df['BRAND'].str.contains('Total')].index)
model_df['E-D(CC)'] = [int(_) for _ in model_df['E-D(CC)']]
model_df['PRICE'] = [int(_) for _ in model_df['PRICE']]
model_df

,YEAR,MONTH,BRAND,MODEL,E-D(CC),PRICE,SALES,SHARES(%)
0,2016,1,Audi,A1 30 TDI,1598,32700,0,0.00
1,2016,1,Audi,A1 Sportback 30 TDI,1598,33700,0,0.00
2,2016,1,Audi,A3 25 TDI,1598,37500,0,0.00
3,2016,1,Audi,A3 35 TDI,1968,38900,101,0.62
4,2016,1,Audi,A3 35 TFSI,1798,42900,3,0.02
...,...,...,...,...,...,...,...,...
36412,2021,3,Volvo,XC60 T8 AWD,1969,83200,186,0.68
36413,2021,3,Volvo,XC90 B6 AWD,1969,82600,21,0.08
36414,2021,3,Volvo,XC90 D5 AWD,1969,80300,0,0.00
36415,2021,3,Volvo,XC90 T6 AWD,1969,95500,0,0.00


In [2]:
# 브랜드 개수 26개
br_ls = model_df['BRAND'].unique()
print(len(br_ls))
br_ls

26


array(['Audi', 'BMW', 'Bentley', 'Cadillac', 'Chrysler', 'Citroen',
       'Fiat', 'Ford', 'Honda', 'Infiniti', 'Jaguar', 'Lamborghini',
       'Land Rover', 'Lexus', 'MINI', 'Mercedes-Benz', 'Nissan',
       'Peugeot', 'Porsche', 'Rolls-Royce', 'Toyota', 'Volkswagen',
       'Volvo', 'Maserati', 'Chevrolet', 'Lincoln'], dtype=object)

In [3]:
pv0 = pd.pivot_table(model_df, index=['BRAND','YEAR','MONTH'],
                    aggfunc={'E-D(CC)':'mean', 'PRICE':'mean', 'SALES':sum, 'SHARES(%)':sum}).round(2).reset_index()
pv0

,BRAND,YEAR,MONTH,E-D(CC),PRICE,SALES,SHARES(%)
0,Audi,2016,1,2821.61,94668.78,1900,11.70
1,Audi,2016,2,2778.38,94015.97,984,6.27
2,Audi,2016,3,2778.38,94015.97,2552,10.56
3,Audi,2016,4,2768.35,92449.40,2474,13.86
4,Audi,2016,5,2766.15,93455.50,2336,12.05
...,...,...,...,...,...,...,...
1500,Volvo,2020,11,1969.00,66340.00,1267,4.61
1501,Volvo,2020,12,1969.00,66340.00,1352,4.30
1502,Volvo,2021,1,1969.00,67842.11,1198,5.36
1503,Volvo,2021,2,1969.00,67842.11,1202,5.39


In [4]:
# 브랜드별 연도별 피벗
# MONTH-해당 브랜드의 매출이 발생한 개월 수
# price-해당 기간 동안 팔린 차종의 평균 단가
# SALES-해당 기간 동안 팔린 대수
# SHARES(%)-해당 브랜드 월 평균 점유율

pv1 = pd.pivot_table(pv0, index=['BRAND', 'YEAR'], 
                      aggfunc={'MONTH':'count', 'E-D(CC)':'mean', 'PRICE':'mean', 'SALES':sum, 'SHARES(%)':'mean'},).round().reset_index()
pv1['AVG_SALES'] = round(pv1['SALES']/pv1['MONTH'])
pv1

,BRAND,YEAR,E-D(CC),MONTH,PRICE,SALES,SHARES(%),AVG_SALES
0,Audi,2016,2784.0,12,94586.0,16718,7.0,1393.0
1,Audi,2017,2859.0,12,99543.0,962,0.0,80.0
2,Audi,2018,2876.0,12,100692.0,12450,5.0,1038.0
3,Audi,2019,2344.0,12,77528.0,12072,5.0,1006.0
4,Audi,2020,2299.0,12,79769.0,26069,9.0,2172.0
...,...,...,...,...,...,...,...,...
140,Volvo,2017,2024.0,12,58556.0,6604,3.0,550.0
141,Volvo,2018,2008.0,12,60474.0,8524,3.0,710.0
142,Volvo,2019,1969.0,12,63147.0,10544,4.0,879.0
143,Volvo,2020,1969.0,12,69430.0,14865,6.0,1239.0


### 브랜드별 연도별 월평균 판매량
- 2021년이 3개월 분량만 있으므로 월평균 판매량으로 시각화
- 벤츠가 기간내 전체 1위
- 2위인 BMW: 2019년 잇따른 화재 사고로 판매량 감소
- 3,4위인 아우디, 폭스바겐: 2017년 환경부 인증 문제로 판매 중단 
- E-D(CC)는 2017년부터 감소세: 환경부 인증 이슈 및 엔진 고성능화를 원인으로 판단

In [7]:
px.line(pv1, x='YEAR', y='AVG_SALES', color='BRAND')

In [8]:
# E-D(CC) 는 2017년을 기점으로 꾸준히 감소 추세
px.line(pv1.groupby('YEAR').mean(), y='E-D(CC)')

### 벤츠를 파보자

#### 우리가 벤츠를 고른 이유
- 시장 점유율 28.1%

In [9]:
# 우리 조가 벤츠를 고른 이유: 판매량의 28.1% 차지
# 브랜드별 pv - 점유율 내림차순
pv2 = pd.pivot_table(pv1,
                     index=['BRAND'], 
                     aggfunc={'E-D(CC)':np.mean,
                              'PRICE':np.mean,
                              'SALES':sum,
                              'SHARES(%)':np.mean}).round(2).reset_index()
pv2 = pv2.sort_values(by=['SHARES(%)'], ascending=False)
fig = go.Figure(data=go.Pie(labels=pv2['BRAND'], values=pv2['SALES'], pull=[0.1]))
fig.show()

#### 벤츠는 수입자동차 중에서 어느 정도 위치인가
- 평균 단가 순위 7위
- 단가 상위 25% 수준(3분위수)
- 랜드로버, 재규어, BMW와 비슷한 수준

In [10]:
# 브랜드별 단가
fig1 = go.Figure()
fig1.add_trace(
    go.Bar(
        x=pv2.sort_values(by='PRICE', ascending=False)['BRAND'],
        y=pv2.sort_values(by='PRICE', ascending=False)['PRICE']),)
fig1.add_trace(
    go.Box(
        y=pv2['PRICE'],),)

#### 월별 추이
- 차를 계절적 요인으로 구매하지는 않으므로 seasonality 규칙은 찾기 어려움
- seasonality 보다는 외부적요인에 의해 구매 여부가 결정
- 굳이 추이를 보기위해 월별 평균값을 보자면, <b>6월과 4분기</b>가 판매량이 제일 많다

In [11]:
# 월별 판매량으로 seasonality를 보려했으나 일정한 규칙이 보이진 않았다
# seasonality 보다는 외부적요인에 의해 구매 여부가 결정되는 듯 하다.
benz0 = pv0[pv0['BRAND']=='Mercedes-Benz']
px.line(benz0, x='MONTH', y='SALES', line_group='YEAR', color='YEAR')

In [12]:
# 굳이 추이를 보기위해 월별 평균값 추이를 구해보면
# 6월과 4분기가 판매량이 제일 많다
px.line(benz0.groupby('MONTH').mean(), y='SALES')

#### 연도별 추이
- 매년 판매량 증가 추세 (21년은 3개월만 있어 제외하고 판단)
- 2018년을 기점으로 cc가 낮아지기 시작 > 환경부 인증 이슈, 엔진 성능 향상 등의 이유
- 점유율은 25~31% 사이, 어차피 매년 점유율 1위
- 평균 판매량이 점유율과 비례하진 않는 것으로 보아, 수입차 시장 규모에 영향을 받는 듯

In [13]:
# SALES 2021년은 3개월만 있으므로 월평균 판매량 컬럼 추가
benz1 = pv1[pv1['BRAND']=='Mercedes-Benz']
benz1['AVG_SALES'] = round(benz1['SALES']/benz1['MONTH'])
benz1

,BRAND,YEAR,E-D(CC),MONTH,PRICE,SALES,SHARES(%),AVG_SALES
97,Mercedes-Benz,2016,3072.0,12,106055.0,56343,25.0,4695.0
98,Mercedes-Benz,2017,3205.0,12,116675.0,68861,30.0,5738.0
99,Mercedes-Benz,2018,3239.0,12,123625.0,70798,27.0,5900.0
100,Mercedes-Benz,2019,2993.0,12,117871.0,76032,31.0,6336.0
101,Mercedes-Benz,2020,2916.0,12,122975.0,81834,30.0,6820.0
102,Mercedes-Benz,2021,2829.0,3,123792.0,19222,27.0,6407.0


In [14]:
# 2021년은 3개월 평균으로 반영되었기에 제외하고 보면 매년 증가 추세
fig2 = make_subplots(rows=3, cols=1)
fig2.add_trace(
    go.Bar(
        x=benz1['YEAR'],y=benz1['AVG_SALES'], name='AVG_SALES'),
        row=1, col=1)
fig2.add_trace(
    go.Scatter(
        x=benz1['YEAR'], y=benz1['E-D(CC)'], name='E-D(CC)'),
        row=2, col=1)
fig2.add_trace(
    go.Scatter(
        x=benz1['YEAR'], y=benz1['SHARES(%)'], name='SHARES(%)'), 
        row=3, col=1)

---

# 고객 Demo별 수입차 구매 데이터

In [15]:
# 데이터 불러오기
demo16 = pd.read_excel("./datas/result_2016.xlsx")
demo17 = pd.read_excel("./datas/result_2017.xlsx")
demo18 = pd.read_excel("./datas/result_2018.xlsx")
demo19 = pd.read_excel("./datas/result_2019.xlsx")
demo20 = pd.read_excel("./datas/result_2020.xlsx")
demo21 = pd.read_excel("./datas/result_2021.xlsx")

In [16]:
demo_df = pd.concat([demo16, demo17, demo18, demo19, demo20, demo21], axis=0, ignore_index=True)
demo_df['Total'] = [int(_.replace(",","")) for _ in demo_df['Total']]
demo_df['year'] = [str(_) for _ in demo_df['year']]
demo_df['age'] = [_.replace("~19","19이하") for _ in demo_df['age']]

### 지역별
- 경기-서울-부산-인천 순으로 구매량이 많음
- 지역별로 선호하는 브랜드가 있다기 보다, 브랜드/지역별 구매량에 비례함
- 연도별 추이를 보았을 때, 지역별 구매량 순위는 변동이 거의 없음
- 벤츠, BMW 가 상위 1,2위를 유지하며 연도별로 큰 차이가 보이지는 않음

In [17]:
area0 = demo_df[demo_df['sales_type'] == '개인']
area1 = area0.groupby(['year','구분','area']).sum().reset_index().sort_values(by='Total', ascending=False)
px.scatter(area1, x='구분', y='area', size='Total', size_max=30)

In [18]:
fig = px.scatter(area0.sort_values(by=['year','area','Total'], ascending=True), x='구분', y="area", animation_frame="year",
           size="Total", color = '구분', size_max=30)
fig.show()

### 연령대
- 주요 구매 연령대 3~40대
- 연도에 따라 인기있는 브랜드는 변화하나, 주요 소비 연령대에는 큰 변화 없음

In [19]:
fig = px.scatter(area0.sort_values(by=['year','age','area','Total'], ascending=True), x='구분', y="area", animation_frame="year",
           size="Total", color = '구분',  hover_name="area",size_max=30,facet_col="age")
fig.show()

### 성별
- 구매량 평균: 남자 48.7K / 여자 19.1K 
- 구매량 여자 대비 남자가 3배
- 30~40대 구매량 압도적으로 많음

In [20]:
sex0 = demo_df.drop(demo_df[demo_df['sales_type'] == '개인'].index).copy()
sex1 = sex0.groupby(['sales_type','age']).sum().reset_index()
sex1

,sales_type,age,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total
0,개인-남자,19이하,8,6,5,4,2,4,7,6,17,16,7,11,93
1,개인-남자,20~29,2311,2697,3426,3684,2809,2966,2487,2713,2814,2791,3536,3088,35322
2,개인-남자,30~39,14935,15433,20753,20928,16551,17217,13930,14211,14550,14918,18681,17660,199767
3,개인-남자,40~49,14958,14405,19219,18374,14745,15911,12659,12528,12507,13440,16095,17297,182138
4,개인-남자,50~59,9912,8971,11425,10649,8693,9688,7667,7263,7273,8024,9537,10625,109727
5,개인-남자,60~69,4677,4100,5212,4838,3721,4304,3231,3027,3023,3451,4183,4889,48656
6,개인-남자,70~,1188,995,1332,1176,933,1174,808,769,823,841,1033,1172,12244
7,개인-여자,19이하,8,4,5,1,1,2,4,5,11,13,6,6,66
8,개인-여자,20~29,1348,1428,1896,1737,1444,1637,1348,1422,1455,1611,1714,1635,18675
9,개인-여자,30~39,7134,7502,9477,9280,7155,7789,6322,6727,6570,7179,8012,8166,91313


In [21]:
px.box(sex1, x='sales_type', y='Total')

In [22]:
px.line(sex1, x='age', y='Total', line_group='sales_type',color='sales_type')